In [9]:
import pandas as pd
import requests
import re
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Cache to store already fetched features
UNIPROT_CACHE = {}
SESSION = requests.Session()

def get_sequence_from_feature(feature, full_sequence, start, end):
    """Extract the actual sequence from the protein sequence"""
    try:
        # First try to get from description
        desc = feature.get('description', '')
        if '(' in desc and ')' in desc:
            match = re.search(r'\(([A-Z]+)\)', desc)
            if match:
                return match.group(1)

        # If not in description, extract from full sequence
        return full_sequence[start-1:end]  # -1 because Python is 0-indexed
    except:
        return None

def get_uniprot_features(uniprot_id):
    """Get binding/active sites for a single UniProt ID"""
    if pd.isna(uniprot_id) or uniprot_id == "Not found":
        return []

    if uniprot_id in UNIPROT_CACHE:
        return UNIPROT_CACHE[uniprot_id]

    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"

    try:
        with SESSION.get(url, timeout=30) as response:
            response.raise_for_status()
            data = response.json()

            sites = []
            full_sequence = data.get('sequence', {}).get('value', '')

            for feature in data.get('features', []):
                if feature['type'] in ('Binding site', 'Active site'):
                    try:
                        location = feature['location']
                        start = location['start']['value']
                        end = location['end']['value']
                        pos = f"{start}" if start == end else f"{start}-{end}"

                        sequence = get_sequence_from_feature(feature, full_sequence, start, end)

                        if sequence:
                            sites.append((pos, sequence))
                    except:
                        continue

            UNIPROT_CACHE[uniprot_id] = sites
            return sites

    except Exception as e:
        print(f"Error fetching {uniprot_id}: {str(e)}")
        return []

def process_csv(input_file, output_file, max_sites=10, max_workers=8):
    """Process CSV and add position/sequence columns"""
    # Read input CSV
    df = pd.read_csv(input_file)

    # Prepare empty columns
    for i in range(1, max_sites+1):
        df[f'Position_{i}'] = pd.NA
        df[f'Sequence_{i}'] = pd.NA

    # Get unique UniProt IDs
    unique_ids = df['UniProt_ID'].dropna().unique()
    print(f"Processing {len(unique_ids)} unique UniProt IDs...")

    # Process IDs with threading
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(get_uniprot_features, uid): uid for uid in unique_ids}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching features"):
            pass

    # Apply to dataframe
    print("Applying features to dataframe...")
    for i in range(1, max_sites+1):
        df[f'Position_{i}'] = df['UniProt_ID'].map(
            lambda x: UNIPROT_CACHE.get(x, [])[i-1][0]
            if x in UNIPROT_CACHE and len(UNIPROT_CACHE[x]) >= i else pd.NA
        )
        df[f'Sequence_{i}'] = df['UniProt_ID'].map(
            lambda x: UNIPROT_CACHE.get(x, [])[i-1][1]
            if x in UNIPROT_CACHE and len(UNIPROT_CACHE[x]) >= i else pd.NA
        )

    # CORRECTED COLUMN ORDERING
    cols = list(df.columns)
    gene_idx = cols.index('GeneSymbol')

    # Get properly sorted columns
    pos_cols = sorted([c for c in cols if c.startswith('Position_')],
                     key=lambda x: int(x.split('_')[1]))
    seq_cols = sorted([c for c in cols if c.startswith('Sequence_')],
                     key=lambda x: int(x.split('_')[1]))

    # Rebuild column order
    cols = [c for c in cols if c not in pos_cols + seq_cols]
    cols[gene_idx+1:gene_idx+1] = pos_cols + seq_cols

    # Save output
    df[cols].to_csv(output_file, index=False)

    elapsed = (time.time() - start_time)/60
    print(f"\nProcessed {len(df)} rows in {elapsed:.2f} minutes")
    print(f"Results saved to {output_file}")
    print(f"Found binding sites for {len(UNIPROT_CACHE)} proteins")

if __name__ == "__main__":
    # Example usage
    process_csv(
        input_file="variant_summary_with_uniprot_ids_updated_file.csv",  # Replace with your CSV file
        output_file="output_with_sites.csv",
        max_sites=20,      # Number of position/sequence pairs to include
        max_workers=8      # Number of concurrent requests
    )

Processing 763 unique UniProt IDs...


Fetching features: 100%|██████████| 763/763 [00:15<00:00, 49.13it/s]

Applying features to dataframe...

Processed 787 rows in 0.26 minutes
Results saved to output_with_sites.csv
Found binding sites for 762 proteins
